In [4]:
import os
import sys
from utils import save_config_to_file

from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


# 여러 모듈 경로를 리스트로 정의합니다.
module_dirs = [
    os.path.abspath('../utils/video/'),
    os.path.abspath('../utils')
]

# 각 모듈 경로를 sys.path에 추가합니다.
for module_dir in module_dirs:
    if module_dir not in sys.path:
        sys.path.append(module_dir)

from split import split_video
from json_utils import create_or_read_json

d:\pose_prediction\data_agent\Video_Understanding\mmaction\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
video_name = "Effective Boxing Combos To Drill.mp4"
video_path = f"D:\\pose_prediction\\data_agent\\data\\video\\{video_name}"
json_path = "../data/json/"

base_name = os.path.splitext(video_name)[0]
folder_path = os.path.join("D:\\pose_prediction\\data_agent\\data\\results", base_name)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder created: {folder_path}")

In [10]:
# data preprocessing

# 모델 이름을 DeepSeek-R1-Distill-Qwen-1.5B로 지정
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Transformers 파이프라인 생성 (생성 파라미터 설정)
pipe = pipeline(
    "text-generation",
    model=model_name,
    trust_remote_code=True,
    max_length=500,      # 최대 생성 길이
    temperature=0.7      # 생성의 다양성 조절
)

# LangChain의 HuggingFacePipeline 래퍼로 변환
llm = HuggingFacePipeline(pipeline=pipe)

prompt_template = PromptTemplate(
    input_variables=["question", "function_result"],
    template=(
        "You are an administrator responsible for data preprocessing and managing video files.\n\n"
        "The video processing pipeline consists of the following steps:\n"
        "1. Split the video into segments. (An executable exists for this task.)\n"
        "2. Run the following executables on the segmented video: alphapose, motionBERT, and PoseC3D. (Executable files for each exist.)\n\n"
        "For each step, please record the success status and the output path in a JSON format.\n\n"
        "The expected JSON structure is:\n"
        "{{\n"
        '  "split_video": {{"success": true/false, "output_path": "string"}},\n'
        '  "alphapose": {{"success": true/false, "output_path": "string"}},\n'
        '  "motionBERT": {{"success": true/false, "output_path": "string"}},\n'
        '  "PoseC3D": {{"success": true/false, "output_path": "string"}}\n'
        "}}\n\n"
        "Existing function result:\n"
        "{function_result}\n\n"
        "User question:\n"
        "{question}\n\n"
        "Provide your detailed answer based on the above."
    ),
    partial_variables={"function_result": ""}
)

chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
questions = [
    "Analyze the JSON context returned by the create_or_read_json function. If any step failed, generate a Python code snippet that debugs the issue; otherwise, indicate the next processing step.",
    "generate a complete and executable Python code snippet that proceeds to the next stage of the video processing pipeline."
]

function_calls = [
    lambda: create_or_read_json(video_name, json_path),
    lambda: split_video(video_path, "../data/video_split", frames_per_chunk=10)
]

In [8]:
split_video(video_path, "../data/video_split", frames_per_chunk=10)

'../data/video_split\Effective Boxing Combos To Drill_chunk1.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk2.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk3.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk4.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk5.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk6.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk7.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk8.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk9.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk10.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk11.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk12.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To Drill_chunk13.mp4' 저장 완료.
'../data/video_split\Effective Boxing Combos To

{'status': 'success',
 'error_code': 0,
 'message': 'D:\\pose_prediction\\data_agent\\data\\video\\Effective Boxing Combos To Drill.mp4: 비디오 분할 완료. 총 408개의 chunk 생성됨.',
 'chunks': ['../data/video_split\\Effective Boxing Combos To Drill_chunk1.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk2.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk3.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk4.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk5.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk6.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk7.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk8.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk9.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk10.mp4',
  '../data/video_split\\Effective Boxing Combos To Drill_chunk11.mp4',
  '../data/video_split\\Effective Boxing Combos T

In [12]:
results_list = []
execution_index = 1

for question, func in zip(questions, function_calls):
    try:
        func_result = func()  # 함수 실행 결과 (예: dict 형태)
        result_str = json.dumps(func_result, ensure_ascii=False, indent=4)
    except Exception as e:
        func_result = {"status": "error", "error_code": 2, "message": str(e), "data": {}}
        result_str = json.dumps(func_result, ensure_ascii=False, indent=4)
    
    # 디버그 모드 여부 판단 및 최종 질문(final_question) 결정
    debug_mode = False
    final_question = question
    if func_result.get("status", "error") != "success":
        debug_mode = True
        final_question = (
            f"The function returned an error with the following details:\n{result_str}\n\n"
            "Please generate a Python code snippet to debug and fix this issue."
        )
    
    # chain.run을 한 번만 호출
    answer = chain.run(question=final_question, function_result=result_str)
    
    # 함수 호출의 소스코드 추출
    try:
        func_source = inspect.getsource(func).strip()
    except Exception as e:
        func_source = f"<<unable to extract source: {e}>>"
    
    # 결과 저장 (실행 순서대로)
    results_list.append({
        "execution_order": execution_index,
        "original_question": question,
        "final_question": final_question,
        "function_call_source": func_source,
        "function_result": result_str,
        "chain_answer": answer,
        "debug_mode": debug_mode
    })
    execution_index += 1
    print(f"Question: {question}\nFinal Question: {final_question}\nAnswer: {answer}\n")

# 최종 구성 정보 생성 및 저장
config = {
    "prompt_template": {
        "template": prompt_template.template,
        "input_variables": prompt_template.input_variables
    },
    "results": results_list
}
save_config_to_file(config, base_filename="my_pipeline_config")

C:\Users\zesky\AppData\Local\Temp\ipykernel_16952\1236481743.py:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  answer = chain.run(question=final_question, function_result=result_str)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Question: Analyze the JSON context returned by the create_or_read_json function. If any step failed, generate a Python code snippet that debugs the issue; otherwise, indicate the next processing step.
Final Question: Analyze the JSON context returned by the create_or_read_json function. If any step failed, generate a Python code snippet that debugs the issue; otherwise, indicate the next processing step.
Answer: You are an administrator responsible for data preprocessing and managing video files.

The video processing pipeline consists of the following steps:
1. Split the video into segments. (An executable exists for this task.)
2. Run the following executables on the segmented video: alphapose, motionBERT, and PoseC3D. (Executable files for each exist.)

For each step, please record the success status and the output path in a JSON format.

The expected JSON structure is:
{
  "split_video": {"success": true/false, "output_path": "string"},
  "alphapose": {"success": true/false, "outpu

'my_pipeline_config_20250219_120739.json'

In [23]:
call_alpha_pose(video_name)

Running command:
['d:\\pose_prediction\\data_agent\\Video_Understanding\\mmaction\\Scripts\\python.exe', 'd:\\pose_prediction\\data_agent\\Video_Understanding\\AlphaPose\\scripts\\demo_inference.py', '--cfg', 'd:\\pose_prediction\\data_agent\\Video_Understanding\\AlphaPose\\configs\\halpe_26\\resnet\\256x192_res50_lr1e-3_1x.yaml', '--checkpoint', 'd:\\pose_prediction\\data_agent\\Video_Understanding\\AlphaPose\\pretrained_models\\halpe26_fast_res50_256x192.pth', '--video', 'D:/pose_prediction/data_agent/data/video/Effective Boxing Combos To Drill.mp4', '--outdir', 'D:\\pose_prediction\\data_agent\\data\\results\\Effective Boxing Combos To Drill.mp4']


{'status': 'error',
 'return_code': 1,
 'stdout': '',
 'stderr': 'Traceback (most recent call last):\n  File "d:\\pose_prediction\\data_agent\\Video_Understanding\\AlphaPose\\scripts\\demo_inference.py", line 13, in <module>\n    from detector.apis import get_detector\nModuleNotFoundError: No module named \'detector\'\n'}

In [2]:
import os
import sys
import subprocess

video_name = "Effective Boxing Combos To Drill.mp4"
# Define the absolute path to the AlphaPose directory (the repository root)
alpha_pose_dir = os.path.abspath("D:/pose_prediction/data_agent/Video_Understanding/AlphaPose")
print("AlphaPose Directory Content:", os.listdir(alpha_pose_dir))

output_dir = os.path.abspath(f"D:/pose_prediction/data_agent/data/output/Effective Boxing Combos To Drill")
video_path = f"D:\\pose_prediction\\data_agent\\data\\video_split\\Effective Boxing Combos To Drill_chunk1.mp4"


# Build full paths to necessary files
demo_inference_script = os.path.join(alpha_pose_dir, "scripts", "demo_inference.py")
cfg_file = os.path.join(alpha_pose_dir, "configs", "halpe_26", "resnet", "256x192_res50_lr1e-3_1x.yaml")
checkpoint_file = os.path.join(alpha_pose_dir, "pretrained_models", "halpe26_fast_res50_256x192.pth")

# Construct the command to run demo_inference.py
command = [
    sys.executable, demo_inference_script,
    '--cfg', cfg_file,
    '--checkpoint', checkpoint_file,
    '--video', video_path,
    '--outdir', output_dir
]

print("Running command:")
print(command)

# Update the environment variables:
env = os.environ.copy()
# Add the AlphaPose root to PYTHONPATH
env["PYTHONPATH"] = alpha_pose_dir + os.pathsep + env.get("PYTHONPATH", "")

# Set the working directory (cwd) to the AlphaPose root
result = subprocess.run(
    command,
    env=env,
    cwd=alpha_pose_dir,
    capture_output=True,
    text=True
)

print("Return Code:", result.returncode)
print("Stdout:", result.stdout)
print("Stderr:", result.stderr)


AlphaPose Directory Content: ['.eggs', '.git', '.gitignore', 'alphapose', 'alphapose.egg-info', 'alphapose_env', 'build', 'configs', 'cython_bbox', 'detector', 'docs', 'examples', 'LICENSE', 'model_files', 'pretrained_models', 'README.md', 'results', 'scripts', 'setup.cfg', 'setup.py', 'trackers']
Running command:
['d:\\pose_prediction\\data_agent\\Video_Understanding\\mmaction\\Scripts\\python.exe', 'D:\\pose_prediction\\data_agent\\Video_Understanding\\AlphaPose\\scripts\\demo_inference.py', '--cfg', 'D:\\pose_prediction\\data_agent\\Video_Understanding\\AlphaPose\\configs\\halpe_26\\resnet\\256x192_res50_lr1e-3_1x.yaml', '--checkpoint', 'D:\\pose_prediction\\data_agent\\Video_Understanding\\AlphaPose\\pretrained_models\\halpe26_fast_res50_256x192.pth', '--video', 'D:\\pose_prediction\\data_agent\\data\\video_split\\Effective Boxing Combos To Drill_chunk1.mp4', '--outdir', 'D:\\pose_prediction\\data_agent\\data\\output\\Effective Boxing Combos To Drill']


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.2800.0_x64__qbz5n2kfra8p0\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\pose_prediction\data_agent\Video_Understanding\mmaction\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.2800.0_x64__qbz5n2kfra8p0\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.2800.0_x64__qbz5n2kfra8p0\lib\subprocess.py", line 1370, in _readerthread
    buffer.append(fh.read())
UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 1891: illegal multibyte sequence


Return Code: 0
Stdout: Loading YOLO model..
Loading pose model from D:\pose_prediction\data_agent\Video_Understanding\AlphaPose\pretrained_models\halpe26_fast_res50_256x192.pth...
===========================> Finish Model Running.
Results have been written to json.

Stderr: None
